<a href="https://colab.research.google.com/github/maxkleiner/maXbox/blob/master/Copy_of_simple_image_classification_with_any_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Simple architecture trained with any dataset
You can learn more about these datasets at:
* CIFAR-10 and CIFAR-100: https://www.cs.toronto.edu/~kriz/cifar.html .
* MNIST: http://yann.lecun.com/exdb/mnist/ .
* FASHION MNIST: https://github.com/zalandoresearch/fashion-mnist .

This example is part of the [K-CAI Neural API](https://github.com/joaopauloschuler/k-neural-api).

In [ ]:
#@title Global Settings
from tensorflow import keras
import tensorflow.keras.layers
import tensorflow.keras.models
import tensorflow.keras.datasets
from tensorflow.keras import regularizers
dataset=tensorflow.keras.datasets.fashion_mnist #@param ["tensorflow.keras.datasets.cifar10", "tensorflow.keras.datasets.cifar100", "tensorflow.keras.datasets.mnist", "tensorflow.keras.datasets.fashion_mnist"] {type:"raw"} 
batch_size=64 # @param [32, 64, 128, 256, 512] {type:"raw"} 
epochs=32 # @param [32, 64, 128, 256, 512] {type:"raw"}
l2_decay = 0.000001 #@param {type:"number"}
initial_learning_rate = 0.01 #@param {type:"number"}
verbose=True #@param {type:"boolean"}
bipolar_input=True #@param {type:"boolean"}

if dataset is keras.datasets.cifar10 or dataset is keras.datasets.cifar100:
  input_shape = (32, 32, 3)
else:
  input_shape = (28, 28, 1)


In [ ]:
import os
if not os.path.isdir('k'):
  !git clone https://github.com/joaopauloschuler/k-neural-api.git k
else:
  !cd k && git pull
!cd k && pip install .

Cloning into 'k'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 356 (delta 38), reused 38 (delta 18), pack-reused 291
Receiving objects: 100% (356/356), 224.47 KiB | 631.00 KiB/s, done.
Resolving deltas: 100% (225/225), done.
Processing /content/k
  Created wheel for cai: filename=cai-0.1.0-cp36-none-any.whl size=21013 sha256=6fd9a9d2042ea91fdb19e1d472f73763455d420c4450364830160266aa3ca0b0
  Stored in directory: /tmp/pip-ephem-wheel-cache-ykytkaa3/wheels/de/4a/3e/5eaf9592f16dae0db5670d85206898522fd9b7a7238081da99
Successfully built cai


In [ ]:
import cai.densenet
import cai.util

save_dir = os.path.join(os.getcwd(), 'saved_models')
cai.util.create_folder_if_required(save_dir)
base_model_name = os.path.join(save_dir, 'simple_model_on_any_dataset')

model = keras.models.Sequential()
model.add(keras.layers.Conv2D(64, (5, 5), padding='valid',
                 input_shape=input_shape, kernel_regularizer=regularizers.l2(l2_decay)) )
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(4, 4)))
model.add(keras.layers.Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(l2_decay)) )
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(l2_decay)) )
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(32, kernel_regularizer=regularizers.l2(l2_decay)) )
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.Dense(32, kernel_regularizer=regularizers.l2(l2_decay)) )
model.add(keras.layers.Activation('relu'))
if (dataset==keras.datasets.cifar100):
  model.add(keras.layers.Dense(100))
else:
  model.add(keras.layers.Dense(10))
model.add(keras.layers.Activation('softmax'))

if (verbose):
    model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 64)        1664      
_________________________________________________________________
activation (Activation)      (None, 24, 24, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 6, 6, 64)          36928     
_________________________________________________________________
activation_1 (Activation)    (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 6, 6, 64)          36928     
_________________________________________________________________
activation_2 (Activation)    (None, 6, 6, 64)          0

In [ ]:
# Learning rate schedule.
def lrscheduler(epoch):
  return initial_learning_rate * (0.99**epoch)

fit_result,  model_name,  csv_name = cai.datasets.train_model_on_dataset(model, dataset, base_model_name, 
  plrscheduler=lrscheduler, batch_size=batch_size, epochs=epochs, lab=False, verbose=verbose, bipolar=bipolar_input)

4423680/4422102 [==============================] - 0s 0us/step
train shape (60000, 28, 28)
test shape (10000, 28, 28)
Loading RGB.
Channel  0  min: -2.0  max: 1.984
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/32

Epoch 00001: val_accuracy improved from -inf to 0.79270, saving model to /content/saved_models/simple_model_on_any_dataset.h5
937/937 - 21s - loss: 0.8354 - accuracy: 0.6847 - val_loss: 0.5361 - val_accuracy: 0.7927 - lr: 0.0100
Epoch 2/32

Epoch 00002: val_accuracy improved from 0.79270 to 0.80620, saving model to /content/saved_models/simple_model_on_any_dataset.h5
937/937 - 22s - loss: 0.5995 - accuracy: 0.7747 - val_loss: 0.5102 - val_accuracy: 0.8062 - lr: 0.0099
Epoch 3/32

Epoch 00003: val_accuracy improved from 0.80620 to 0.84140, saving model to /content/saved_models/simple_model_on_any_dataset.h5
937/937 - 23s - loss: 0.5388 - accuracy: 0.7960 - val_loss: 0.4192 - val_accuracy: 0.8414 - lr: 0.0098
Epoch 4/32

Epoch 00004: val_a

In [ ]:
model.load_weights(model_name)

In [ ]:
from google.colab import files

In [ ]:
files.download(csv_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download(model_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>